Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
IIC2433 - Minería de Datos
<br>

<center>
    <h2> Tarea 5 - Autoencoders </h2>
    <h1>   </h1>
    <p>
        Profesor Marcelo Mendoza<br>
        Segundo Semestre 2022<br>    
        Fecha de entrega: 25 de noviembre
    </p>
    <br>
</center>

<br>

## Indicaciones

Deberás entregar **SOLO** el archivo .ipynb en el buzón respectivo en canvas. 

**IMPORTANTE**: 
- Se te dará puntaje tanto por código como por la manera en la que respondas las preguntas planteadas. Es decir, si tienes un código perfecto pero este no es explicado o no se responden preguntas asociadas a este, no se tendrá el puntaje completo.
- El notebook debe tener todas las celdas de código ejecutadas. Cualquier notebook que no las tenga no podrá ser corregido.
- El carácter de esta tarea es **INDIVIDUAL**. Cualquier instancia de copia resultará en un 1,1 como nota de curso.

#1. Obtención del dataset

Para esta tarea trabajaremos con el dataset [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist). Este contiene múltiples imágenes de artículo de ropa representados como un dato con 28x28 pixeles.




In [1]:
#Librerías que les podrían ser útiles
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

El dataset es parte de la librería de keras, por lo que no necesitan descargarlo. A continuación esta el código para obtenerlo. También para efectos de esta tarea solo trabajaremos con un subset de los datos, para reducir los tiempos de ejecución.

In [2]:
from sklearn.model_selection import train_test_split

(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
X = np.concatenate([x_train, x_test], axis=0)
Y = np.concatenate([y_train, y_test], axis=0)
_, X, _, Y = train_test_split(X, Y, test_size=0.2, random_state=23)
X_matrix = np.expand_dims(X, -1).astype("float32") / 255

#2. Reducción de dimensionalidad (3 pts)

En esta parte usaremos dos métodos diferentes para reducir las dimensionalidades del dataset. Para ambos casos se hará una reducción a 6 dimensionalidades.

Para que los datos sean compatibles con el modelo es necesario transformar de una matriz de 28x28 a una fila con todos los datos.

In [3]:
X_flat = []
for i in range(X.shape[0]):
    X_flat.append(X[i].flatten())
X_flat = np.array(X_flat)

### 2.0 Normalizar (0.25 pts)

Para que las capas funcionen mejor, es necesario normalizar los datos.

In [4]:
from sklearn.preprocessing import MinMaxScaler

In [5]:
# Se define min max scaler
scaler = MinMaxScaler()

#Normaliza
X_flat_df = pd.DataFrame(X_flat)
X_df_n = scaler.fit_transform(X_flat_df)

X_df_n

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

##2.1 PCA (0.75 pts)

Primero debes hacer un PCA de 6 dimensiones. Y luego usar `cross_val_score`, con `cv=5` para medir el `accuracy` promedio de la reducción en una predicción.

In [6]:
from sklearn.decomposition import PCA

In [87]:
#Realiza el PCA
pca_6 = PCA(n_components=6)
X_pca = pca_6.fit_transform(X_df_n)
print(X_pca)
X_pca.shape

[[ 1.72610137 -4.03377579  0.85600161 -0.36006788  0.50919045  0.38765597]
 [ 4.28380382 -4.06065227  2.69805022 -0.10023676  0.07208649  1.50045908]
 [-3.38741289 -0.15876603 -3.66314479  1.79730453 -0.57426369 -0.7864482 ]
 ...
 [ 5.19428138 -1.79200798  1.39094472 -2.08765819  1.53335431 -0.64107263]
 [-2.04807435 -2.57483705 -2.01499749  0.70129716 -0.40156554  0.11188742]
 [ 0.36500625 -1.04644345 -0.71341361 -1.47716135  0.18000252  0.39106875]]


(14000, 6)

Caculamos la varianza

In [8]:
print(pca_6.explained_variance_ratio_)
print(np.mean(pca_6.explained_variance_ratio_))

[0.29009527 0.17671623 0.059695   0.04983527 0.03840709 0.03480205]
0.10825848311100379


Para clasificar usaremos un modelo de Random Forest, el cual luego se debe usar para medir el `accuracy`.

In [9]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=18)

In [10]:
from sklearn.model_selection import cross_val_score
#Realiza la prediccion
accuracy = cross_val_score(rfc, X_pca, Y, cv=5)
print(accuracy)
print(np.mean(accuracy))

[0.75964286 0.75107143 0.75928571 0.75785714 0.75035714]
0.7556428571428572


##2.2 Autoencoders (1 pts)

Ahora debes usar autoencoders para reducir a 6 dimensiones. 
Para esta parte te pedimos que alcances por lo menos un **0.04** de accuracy por sobre el obtenido en PCA utilizando `cross_val_score` de la misma forma. Para lograr esto puedes cambiar los hiperparámetros en diferentes partes, agregar capas, etc. Es importante notar que `X_train` y `X_test` definidos a continuación son para ser usados en el entrenamiento del autoencoder, para la predicción se sigue teniendo que usar el X estandarizado antes obtenido.

In [125]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(X_df_n, test_size=0.3, random_state=23)

In [126]:
print(X_test.shape)
print(X_train.shape)

(4200, 784)
(9800, 784)


Se usó https://pythonprogramming.net/autoencoders-tutorial/, https://towardsdatascience.com/deep-inside-autoencoders-7e41f319999f

In [127]:
input_size = 784
hidden_size_1 = 240
hidden_size_2 = 64
code_size = 6

x = keras.layers.Input(shape=(input_size,))

# Encoder
hidden_1 = Dense(hidden_size_1, activation='selu')(x)
hidden_2 = Dense(hidden_size_2, activation='selu')(hidden_1)
h = Dense(code_size, activation='selu')(hidden_2)

# Decoder
hidden_3 = Dense(hidden_size_2, activation='selu')(h)
hidden_4 = Dense(hidden_size_1, activation='selu')(hidden_3)
r = Dense(input_size, activation='sigmoid')(hidden_4)

autoencoder = keras.Model(x, r, name="autoencoder")
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 784)]             0         
                                                                 
 dense_90 (Dense)            (None, 240)               188400    
                                                                 
 dense_91 (Dense)            (None, 64)                15424     
                                                                 
 dense_92 (Dense)            (None, 6)                 390       
                                                                 
 dense_93 (Dense)            (None, 64)                448       
                                                                 
 dense_94 (Dense)            (None, 240)               15600     
                                                                 
 dense_95 (Dense)            (None, 784)               

In [128]:
autoencoder.fit(X_train, X_train, epochs=150, batch_size=256, validation_data=(X_test, X_test))

Epoch 1/150
39/39 [==============================] - 2s 28ms/step - loss: 0.0855 - val_loss: 0.0461
Epoch 2/150
39/39 [==============================] - 1s 24ms/step - loss: 0.0408 - val_loss: 0.0388
Epoch 3/150
39/39 [==============================] - 1s 23ms/step - loss: 0.0357 - val_loss: 0.0350
Epoch 4/150
39/39 [==============================] - 1s 22ms/step - loss: 0.0331 - val_loss: 0.0332
Epoch 5/150
39/39 [==============================] - 1s 22ms/step - loss: 0.0314 - val_loss: 0.0313
Epoch 6/150
39/39 [==============================] - 1s 23ms/step - loss: 0.0291 - val_loss: 0.0285
Epoch 7/150
39/39 [==============================] - 1s 23ms/step - loss: 0.0272 - val_loss: 0.0271
Epoch 8/150
39/39 [==============================] - 1s 23ms/step - loss: 0.0262 - val_loss: 0.0264
Epoch 9/150
39/39 [==============================] - 1s 23ms/step - loss: 0.0255 - val_loss: 0.0258
Epoch 10/150
39/39 [==============================] - 1s 23ms/step - loss: 0.0250 - val_loss: 0.0253

In [129]:
X_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [130]:
prediccion = autoencoder.predict(X_df_n)

438/438 [==============================] - 2s 3ms/step


In [131]:
rfc = RandomForestClassifier()

In [132]:
#Usa cross_val_score para predecir con rfc
accuracy = cross_val_score(rfc, prediccion, Y , cv=5)
print(accuracy)
print(np.mean(accuracy))

[0.825      0.8175     0.8325     0.82285714 0.81964286]
0.8234999999999999


Podemos ver que el accuracy es más de 0.04 mejor que en PCA

##2.3 Preguntas (1 pto)

1. ¿Por qué el Autoencoder tiene un encoder y decoder?

El encoder realiza la función de comprimir las variables y reducir la dimensionalidad, mientras que el decoder toma los datos con la dimensionalidad reducida y los vuelve a dejar en la dimensionalidad inicial.

2. ¿Por qué es posible obtener mejores resultados usando Autoencoders por sobre PCA para reducir dimensiones?

Se utilizó https://towardsdatascience.com/autoencoders-vs-pca-when-to-use-which-73de063f5d7

PCA es escencialmente una transformación lineal, en cambio autoencoder es capaz de modelar funciones no lineares complejas. Otra diferencia es que al ser entrenado, el autoencoder mantiene cierta correlación que PCA no. El hecho de que se pueda agregar layers al autoencoder le agrega complejidad y certeza, cosa que no puede hacer el PCA

#3. Autoencoder variacional (3 pts)

Los autoencoders variacionales (VAEs), son una potente herramienta que permite incorporar el uso de estadísticas en autoencoders tradicionales, y permiten identificar de muy buena forma dimensiones latentes en un dataset.

Tu tarea en esta sección es implementar un VAE que trabaje sobre el mismo dataset de imagenes de ropa de la parte anterior. Para esto, es fuertemente recomendado que utilices la documentación de [Keras](https://keras.io/) como guía para la implementación.

Disclaimer: Es posible que tu código en esta parte se demore en correr, pero debería tomar a lo más 1 hora.

##3.1 VAE (1.5 pts)

Se utilizó https://towardsdatascience.com/variational-autoencoders-as-generative-models-with-keras-e0c79415a7eb

In [138]:
#Encoder

input_data = keras.layers.Input(shape=(28, 28, 1))
encoder = keras.layers.Conv2D(64, (5,5), activation='relu')(input_data)
encoder = keras.layers.MaxPooling2D((2,2))(encoder)
encoder = keras.layers.Conv2D(64, (3,3), activation='relu')(encoder)
encoder = keras.layers.MaxPooling2D((2,2))(encoder)
encoder = keras.layers.Conv2D(32, (3,3), activation='relu')(encoder)
encoder = keras.layers.MaxPooling2D((2,2))(encoder)
encoder = keras.layers.Flatten()(encoder)
encoder = keras.layers.Dense(6)(encoder)

In [143]:
def sample_latent_features(distribution):
    distribution_mean, distribution_variance = distribution
    batch_size = tf.shape(distribution_variance)[0]
    random = keras.backend.random_normal(shape=(batch_size, tf.shape(distribution_variance)[1]))
    return distribution_mean + tf.exp(0.5 * distribution_variance) * random
 
distribution_mean = keras.layers.Dense(2, name='mean')(encoder)
distribution_variance = keras.layers.Dense(2, name='log_variance')(encoder)
latent_encoding = keras.layers.Lambda(sample_latent_features)([distribution_mean, distribution_variance])

In [144]:
encoder_model = tf.keras.Model(input_data, latent_encoding)
encoder_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d_2 (Conv2D)              (None, 24, 24, 64)   1664        ['input_21[0][0]']               
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 12, 12, 64)   0           ['conv2d_2[0][0]']               
                                                                                                  
 conv2d_3 (Conv2D)              (None, 10, 10, 64)   36928       ['max_pooling2d[0][0]']          
                                                                                            

In [145]:
#Decoder
decoder_input = keras.layers.Input(shape=(2))
decoder = keras.layers.Dense(64)(decoder_input)
decoder = keras.layers.Reshape((1, 1, 64))(decoder)
decoder = keras.layers.Conv2DTranspose(64, (3,3), activation='relu')(decoder)
 
decoder = keras.layers.Conv2DTranspose(64, (3,3), activation='relu')(decoder)
decoder = keras.layers.UpSampling2D((2,2))(decoder)
 
decoder = keras.layers.Conv2DTranspose(64, (3,3), activation='relu')(decoder)
decoder = keras.layers.UpSampling2D((2,2))(decoder)
 
decoder_output = keras.layers.Conv2DTranspose(1, (5,5), activation='relu')(decoder)

In [146]:
decoder_model = keras.Model(decoder_input, decoder_output)
decoder_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 2)]               0         
                                                                 
 dense_97 (Dense)            (None, 64)                192       
                                                                 
 reshape_22 (Reshape)        (None, 1, 1, 64)          0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 3, 3, 64)         36928     
 nspose)                                                         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 5, 5, 64)         36928     
 ranspose)                                                       
                                                                 
 up_sampling2d (UpSampling2D  (None, 10, 10, 64)       0   

In [147]:
#VAE    
encoded = encoder_model(input_data)
decoded = decoder_model(encoded)
autoencoder = keras.models.Model(input_data, decoded)
autoencoder.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 model_1 (Functional)        (None, 2)                 57282     
                                                                 
 model_2 (Functional)        (None, 28, 28, 1)         112577    
                                                                 
Total params: 169,859
Trainable params: 169,859
Non-trainable params: 0
_________________________________________________________________


In [151]:
autoencoder.compile(optimizer='adam', loss='mse')

Para esta parte te recomendamos usar los datos en la forma `X_matrix` definida en la obtención del dataset.

In [154]:
#Entrenamiento VAE
autoencoder.fit(X, Y, epochs=2, batch_size=64, validation_data=(x_test, x_test))

Epoch 1/2
219/219 [==============================] - 39s 169ms/step - loss: 27.8861 - val_loss: 13427.7100
Epoch 2/2
219/219 [==============================] - 38s 174ms/step - loss: 27.8666 - val_loss: 13427.7168


Ahora que tienes un VAE entrenado para este dataset, muestra su espacio latente correspondiente.

In [158]:
#Espacio latente
x = []
y = []
z = []
for i in range(10000):
    z.append(y_test[i])
    op = encoder_model.predict(np.array([Y[i]]))
    x.append(op[0][0])
    y.append(op[0][1])
df = pd.DataFrame()
df['x'] = x
df['y'] = y
df['z'] = ["digit-"+str(k) for k in z]
plt.figure(figsize=(8, 6))
sns.scatterplot(x='x', y='y', hue='z', data=df)
plt.show()

ValueError: in user code:

    File "c:\users\francisco\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "c:\users\francisco\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\users\francisco\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "c:\users\francisco\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py", line 2079, in predict_step
        return self(x, training=False)
    File "c:\users\francisco\appdata\local\programs\python\python37\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
   

##3.2 Preguntas teóricas (1.5 pts)

1. ¿En qué parte del código se puede encontrar la diferencia principal entre un AE tradicional y VAE? (Copia y pega la parte del código para explicar mejor)

Se puede ver que al momento de definir las layers hay otras funciones tales como Conv2DTranspose, UpSampling2D para el decode y Conv2D, MaxPooling2D para el encode. Además se crea la siguiente funcion:

2. Explica qué hace la parte del código de la pregunta anterior.

Esta función completa el encoder sacando el promedio y la log-varianza para para formar un latent encoding vector. Este es pasado al decoder con el fin de reconstruir la data.

3. ¿En qué situación puede ser útil utilizar VAE y por qué? (Da 2 ejemplos)

Se uso https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8774760/

VAE es util cuando la data que uno tiene debe ser reconstruida lo más cercano a lo original. Algunos ejemplos son generar data para audios, textos e imagenes.

4. ¿Cúal es una posible desventaja del método VAE?

Toma mucho tiempo y recursos computacionales, Por lo que si el problema puede ser resuelto con AE mejor que se resuelva con eso.